In [1]:
cd ..

/gorani/gorani/backend/dataserver


In [2]:
# parameters
user_id = 1
dropout = 0.2
neuron = 64
outlie_time = 200

In [3]:
from gorani.tf import FEATURE_LEN, transform_paragraphs
from gorani.datadb import DataDB

data_db = DataDB()
ps = data_db.get_flipped_paragraphs(user_id)

x,y = transform_paragraphs(ps)
print(len(x), len(y))

4 4


In [4]:
import numpy as np
idx = np.where(y > outlie_time)
print(len(idx))
x = np.delete(x, idx, axis=0)
y= np.delete(y, idx, axis=0)
print(x.shape, y.shape)
print(x.dtype, y.dtype)

2
(2, 268, 44) (2, 1)
float32 float32


In [5]:
import seaborn as sns
sns.set(color_codes=True)
sns.countplot(x[:, :, FEATURE_LEN - 1].flatten())

In [6]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, shuffle= True)
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)
print(x_train.dtype, y_train.dtype)
print(FEATURE_LEN)

(1, 268, 44) (1, 1) (1, 268, 44) (1, 1)
float32 float32
44


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Input, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as K
from typing import List
from gorani.tfmodels import time_model
import time

with tf.Graph().as_default():
    with tf.Session() as sess:
        K.set_session(sess)
        model = time_model(FEATURE_LEN, neuron, dropout)
        model.compile(loss='mse', optimizer='adagrad')
        filepath = "out/{}-time-model.hdf5".format(user_id)
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        tensorboard = TensorBoard(log_dir='out/logs/{}-{}-{}-time-model-{}'.format(user_id, neuron, dropout, time.time()), histogram_freq=1,
                                  write_graph=True, write_images=False)

        history = model.fit(x_train, y_train, epochs=50, batch_size=int(x_train.shape[0]/20+1), verbose=1, validation_data=(x_valid, y_valid), callbacks=[checkpoint, tensorboard])


Train on 1 samples, validate on 1 samples
Epoch 1/50

Epoch 00001: val_loss improved from inf to 83.25948, saving model to out/1-time-model.hdf5
1/1 [==============================] - 1s 748ms/step - loss: 6.4234 - val_loss: 83.2595
Epoch 2/50

Epoch 00002: val_loss improved from 83.25948 to 82.66955, saving model to out/1-time-model.hdf5
1/1 [==============================] - 0s 217ms/step - loss: 0.2943 - val_loss: 82.6695
Epoch 3/50

Epoch 00003: val_loss did not improve from 82.66955
1/1 [==============================] - 0s 206ms/step - loss: 0.1076 - val_loss: 83.2570
Epoch 4/50

Epoch 00004: val_loss improved from 82.66955 to 82.58733, saving model to out/1-time-model.hdf5
1/1 [==============================] - 0s 232ms/step - loss: 0.2261 - val_loss: 82.5873
Epoch 5/50

Epoch 00005: val_loss did not improve from 82.58733
1/1 [==============================] - 0s 210ms/step - loss: 0.0974 - val_loss: 83.2530
Epoch 6/50

Epoch 00006: val_loss improved from 82.58733 to 82.47087, s